The calculate spot heights notebook estimates a height above nearest neighbour by interogating the outputs of a hydraulic model at points upstream and downstream of a reach.

Graphs of the water levels at the points are generated as a biproduct of this process

It produces a file hand-levels.csv with the new levels. If you already have appropriate hand levels, then this step is not necessary (and will overwrite the values you already have).

In [ ]:
import os
import io
from dotenv import load_dotenv
load_dotenv()
import logging
import cmcrameri.cm as cmc
import numpy

logging.getLogger().setLevel('WARNING')
output_folder = os.environ['OUTPUT_FOLDER']
hand_points_shape = os.environ['HAND_POINTS']
zone_definition_albers_shape = os.environ['ZONE_DEF_ALBERS']
highres = os.environ['HIGHRES'] == 'TRUE'
if highres:
    dpi=600
    hextent="_high.tif"
    fig_args = {'format': "tiff", 'pil_kwargs': {"compression": "tiff_lzw"}}
else:
    dpi=300
    hextent=".png"
    fig_args = {'format':"png"}


#colour_scheme = cmc.batlowS.colors
colour_scheme = numpy.array([[0x1F, 0x77, 0xB4], [0xF2, 0x71, 0x07], [
                            0xAA, 0xE0, 0x07], [0x8C, 0x56, 0x4B], [0xDB, 0x72, 0xBC]])/256.0


Note: Ignore "ERROR:root:Hand level was not found for date 2016-08-18 in Murray - GKPF - 2016-07" when run the first time (or update hand point file)

In [ ]:
from hydrological_connectivity.definitions.definitions_generator_factory import DefinitionsGeneratorFactory
from rasterio.windows import from_bounds
import rasterio
from hydrological_connectivity.processing.compare_flood_rasters_rasterio import CompareFloodRastersRasterIo
from hydrological_connectivity.processing.compare_flood_rasters_elev_rasterio import CompareFloodRastersElevRasterIo
from hydrological_connectivity.processing.calculate_hydraulic_level import CalculateHydraulicLevel
import logging
import geopandas
logging.getLogger().setLevel('INFO')
definition = DefinitionsGeneratorFactory.get_generator()

definition.generate()

rows_points = geopandas.read_file(hand_points_shape).transpose()

rows_regions = geopandas.read_file(zone_definition_albers_shape).transpose()

depth_results = {}
se_results = {}
for segment_index in rows_points:
    row_pt = rows_points[segment_index]
    segment_id = row_pt.loc['Segment_Id']
    description = row_pt.loc['Descript']
    geometry = row_pt.loc['geometry']
    if segment_id not in rows_regions:
        logging.warning(f"Hand point {description} skipped")
        continue
    row_region = rows_regions[segment_id]

    short_location_name = row_region['Short_Loc']
    region = row_region.loc['Region']
    logging.info(f"processing {short_location_name}")
    depth_segment_results = {}
    se_segment_results = {}

    if not(region in definition.models):
        logging.warning(
            f"Region '{region}' was not found in the list of models, SKIPPING")
        continue

    hydraulic_model = definition.models[region]

    contiguous_ranges = hydraulic_model.get_contiguous_date_ranges()

    geometry_points = [(geometry.x, geometry.y)]
    display(geometry_points)
    for (date_rec, depth_raster_location) in hydraulic_model.depth_outputs.items():
        range_index = 0
        for contiguous in contiguous_ranges:
            if (date_rec >= contiguous['start'] and date_rec <= contiguous['end']):
                break
            range_index = range_index+1

        if range_index not in depth_segment_results:
            depth_segment_results[range_index] = {}
            se_segment_results[range_index] = {}

        range_results = depth_segment_results[range_index]
        se_range_results = se_segment_results[range_index]

        surface_elevation_raster = hydraulic_model.elevation_outputs[date_rec]
        elevation_raster = definition.global_elevation.elevation_filename
        logging.info(f"Elevation raster {elevation_raster}")

        if range_index == 0 and description == "US SA - Weir Pool 5":
            offset_geometry = [(geometry.x+155/110000, geometry.y)]
            logging.info(
                f"Special case {elevation_raster} {offset_geometry}")
            surface_elevation = CalculateHydraulicLevel(
                surface_elevation_raster, offset_geometry)
        else:
            surface_elevation = CalculateHydraulicLevel(
                surface_elevation_raster, geometry_points)

        dem_elevation = CalculateHydraulicLevel(
            elevation_raster, geometry_points)
        depth = CalculateHydraulicLevel(depth_raster_location, geometry_points)

        try:
            surface_elevation.execute()
            dem_elevation.execute()
            depth.execute()
            heights = depth.heights
            if len(depth.heights) > 0:
                if len(depth.heights[0]) > 0:
                    heights = depth.heights[0][0]

            surface_elevation_heights = surface_elevation.heights
            if len(surface_elevation.heights) > 0:
                if len(surface_elevation.heights[0]) > 0:
                    surface_elevation_heights = surface_elevation.heights[0][0]

            dem_elevation_heights = dem_elevation.heights
            if len(dem_elevation.heights) > 0:
                if len(dem_elevation.heights[0]) > 0:
                    dem_elevation_heights = dem_elevation.heights[0][0]

            logging.info(
                f"observed level of {heights}m for zone {segment_index} for file {depth_raster_location} - using elevation {surface_elevation_heights} vs {dem_elevation_heights}m")
            se_range_results[(date_rec,
                            surface_elevation_raster)] = surface_elevation_heights - dem_elevation_heights
            range_results[(date_rec,
                        depth_raster_location)] = heights
        except:
            logging.exception(
                f"Problem with one or more of these files: {(depth_raster_location,surface_elevation_raster,elevation_raster)}")
    depth_results[segment_index] = depth_segment_results
    se_results[segment_index] = se_segment_results


In [ ]:
results = se_results
#results = depth_results

In [ ]:
import pickle
from pathlib import Path
import os

Path(output_folder ).mkdir(parents=True, exist_ok=True)


output = open(output_folder + f'{os.path.sep}hand_levels.pkl', 'wb')
pickle.dump(results, output)
output.close()


In [ ]:
import pickle

input_file = open(f'{output_folder}{os.path.sep}hand_levels.pkl', 'rb')
results = pickle.load(input_file)
input_file.close()


In [ ]:
import numpy
import pandas
flattened = []

rows_points = geopandas.read_file(hand_points_shape).transpose()

rows_regions = geopandas.read_file(zone_definition_albers_shape).transpose()

for (zone, internals) in results.items():
    for (contiguous_range, result_sec) in internals.items():
        dates = []
        volumes = []
        for ((date_t, file_t), volume) in result_sec.items():
            dates.append(date_t)
            if (volume == numpy.finfo(numpy.float64).min):
                volumes.append(numpy.nan)
            else:
                volumes.append(volume)

        first_date = numpy.min(dates)

        dataframe = (pandas.DataFrame(volumes, index=dates, columns=[
                     f'{rows_points[zone]["Descript"]} {first_date:%Y-%m}']))
        flattened.append(dataframe)

result_set = pandas.concat(flattened, join='outer', axis=1)


In [ ]:
import pkgutil
peak_event_df = pandas.read_csv(
    f"{os.environ['INPUT_FOLDER']}{os.path.sep}peak-events.csv", index_col=0, parse_dates=[1])
peak_events = peak_event_df.to_dict()['PeakEvent']


In [ ]:
import numpy
x = {long_label: long_label[3:] for long_label in result_set.columns}
swapped = {}
for k, v in x.items():
    if v not in swapped:
        swapped[v] = set()
    swapped[v].add(k)

merged_series = {short_label: result_set[set_of_series].mean(axis=1, skipna=True) for short_label,
                 set_of_series in swapped.items()}


In [ ]:
from matplotlib import pyplot
import seaborn
import logging
import matplotlib.dates as mdates

#seaborn.set_theme(style="whitegrid")
seaborn.set(font_scale=0.75)
pyplot.style.use('seaborn-white')

fig, axes_list = pyplot.subplots(
    nrows=7, ncols=4, figsize=((8.3), (11.7-2)), sharey='row', dpi=dpi, gridspec_kw={'left': 0.045, 'right': 0.99, 'top': 0.97, 'bottom': 0.02, 'wspace': 0.05, 'hspace': 0.2})  # gridspec_kw={'left': 0.01, 'right': 1, 'top': 1, 'bottom': 0, 'wspace': 0.05, 'hspace': 0.05})

fig.delaxes(ax=axes_list[6,2])
fig.delaxes(ax=axes_list[6,3])

grid_found=[]

filtered_result_set = pandas.DataFrame(
    result_set[[x for x in result_set.columns if '1956' not in x]])

display_order = [22, 23, 24, 25, 0, 1, 2, 3, 4, 5, 6, 7,
                 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 16, 17, ]

# Ugly code to get the row-wise maximumum
row_max_list = {j: [] for j in range(7)}
for index in range(0, len(filtered_result_set.columns)):
    i = index

    col_lookup = filtered_result_set.columns[i][3:]
    original = True
    if col_lookup not in grid_found:
        grid_found.append(col_lookup)
        original = False

    g_index = grid_found.index(col_lookup)
    d_index = display_order.index(g_index)
    result_array = filtered_result_set[filtered_result_set.columns[i]].loc[~numpy.isnan(
        filtered_result_set[filtered_result_set.columns[i]])]
    row_max_list[d_index//4].append(numpy.max(result_array))
    #print((index, g_index, d_index, col_lookup, numpy.max(result_array)))

row_max = {j: numpy.max(v, initial=0) for j, v in row_max_list.items()}

for index in range(0, len(filtered_result_set.columns)):
    i = index

    col_lookup = filtered_result_set.columns[i][3:]
    original=True
    if col_lookup not in grid_found:
        grid_found.append(col_lookup)
        original=False

    g_index = grid_found.index(col_lookup)
    d_index = display_order.index(g_index)
    axis = axes_list[d_index // 4][d_index % 4]
    #display(f'{col_lookup}: {g_index}/{d_index}: ({d_index // 4},{d_index % 4})')
    
    result_array = filtered_result_set[filtered_result_set.columns[i]].loc[~numpy.isnan(
        filtered_result_set[filtered_result_set.columns[i]])]

    prod_desc = filtered_result_set.columns[i]

    short_label = prod_desc[3:]
    mean_series = merged_series[short_label]
    graphical_max = row_max[d_index // 4]
    seaborn.set_palette(colour_scheme)
    if len(result_array) > 1:
        axis.set_prop_cycle(color=[colour_scheme[0]] if f"{filtered_result_set.columns[i][0:2]}"=='US' else [colour_scheme[1]])
        max_value = numpy.max(numpy.max(filtered_result_set[[x for x in numpy.max(
            filtered_result_set).index if x[3:] == col_lookup]]))
        max_index = numpy.argmax(
            filtered_result_set[filtered_result_set.columns[i]])
        max_value = graphical_max
        res = seaborn.lineplot(x="index", y=filtered_result_set.columns[i], 
        data=filtered_result_set.reset_index(),
                               ax=axis)
        if col_lookup in peak_events:
            axis.vlines(peak_events[col_lookup], ymin=0,
                        ymax=max_value*1.1, linewidth=1., color=colour_scheme[2])
            logging.debug(
                f"Found '{filtered_result_set.columns[i]}' with value {peak_events[col_lookup]}")
        else:
            logging.error(
                f"Did not find '{col_lookup}' derived from '{filtered_result_set.columns[i]}'")

        axis.vlines(filtered_result_set.index[max_index], ymin=0,
                    ymax=max_value*1.1, linewidth=1., color='grey', linestyles='dashed')

        axis.set_ylim(ymin=0)
        res.set_ylabel(None)
        res.set_xlabel(None)
        res.set_xticklabels([])
        #locator = mdates.AutoDateLocator(interval_multiples=False)
        #axis.xaxis.set_major_locator(locator)
        #pyplot.setp(axis.get_xticklabels(), rotation=45, ha="right")   # optional
        res.set_title(short_label)
    elif len(result_array) == 1:
        #data_frame = pandas.DataFrame.from_dict({'x': [f"{ft:%Y-%m-%d}" for ft in result_array.index], 'y': result_array.values,
        #                                         'hue': [f"{filtered_result_set.columns[i][0:2]}"]})
        #res = seaborn.barplot(data=data_frame, ax=axis)
        res = seaborn.barplot(x=[f"{ft:%Y-%m-%d}" for ft in result_array.index], y=result_array.values,
                              hue=[f"{filtered_result_set.columns[i][0:2]}"], hue_order=['US', 'DS'],
                              ax=axis, palette=colour_scheme)

        j=0
        us_ds_label = {0: 'U/S', 1: 'D/S', 2: 'U/S', 3: 'D/S'}
        for p in res.patches:
            res.annotate(us_ds_label[j], (p.get_x() + p.get_width() / 2., p.get_height()),
                           ha='center', va='center', fontsize=6, color='black', xytext=(0, -10),
                           textcoords='offset points')
            j=j+1

        res._remove_legend(res.get_legend())

        res.set_title(short_label)
        res.set_ylabel(None)
        res.set_xlabel(None)
        res.set_xticklabels([])

        if col_lookup in peak_events:
            chosen_date = peak_events[col_lookup]
            level = mean_series.loc[chosen_date]
        else:
            chosen_date = mean_series.index[mean_series.argmax()]
            level = mean_series.loc[chosen_date]
    
        #axis.text(f"{chosen_date:%Y-%m-%d}", level, f"{level:02}")

fig.suptitle("Water level (m)",
             x=0.01, y=0.5, ha='center', va='center', rotation=90)

fig.text(0.5, 0.01, "Date range as per model", ha='center', va='center')
#fig.subplots_adjust(left=0.5)
#fig.autofmt_xdate()
#pyplot.tight_layout(rect=[0, 0, 1, 1])
volume_file_name = output_folder + f"{os.path.sep}fig1_supplement_hand_points" + hextent
pyplot.savefig(volume_file_name, **fig_args)


In [ ]:
from datetime import timedelta 
hand_params=[]
for short_label, mean_series in merged_series.items():
    for segment_index in rows_regions:
        row_region = rows_regions[segment_index]
        short_location_name = row_region['Short_Loc']
        if short_location_name in short_label:
            break
                  
    if short_label in peak_events:
        chosen_date = peak_events[short_label]
        level = mean_series.loc[chosen_date]
        hand_params.append(
            {
                "image": f"{short_location_name}",
                "date": f"{chosen_date:%Y-%m-%d}",
                "level": f"{level:0.2f}"
            }
        )
        # print(f'{short_label} "{chosen_date}" "{level}"')
    else:
        chosen_date = mean_series.index[mean_series.argmax()] 
        level = mean_series.loc[chosen_date]
        chosen_date = chosen_date+ pandas.DateOffset(days=1)
        # print(f'*{short_label} "{chosen_date}" "{level}"')
        hand_params.append(
            {
                "image": f"{short_location_name}",
                "date": f"{chosen_date:%Y-%m-%d}",
                "level": f"{level:0.2f}"
            }
        )


In [ ]:
pandas.DataFrame(hand_params).sort_values(["image","date"])


In [ ]:
pandas.DataFrame(hand_params).sort_values(["image", "date"]).to_csv(
    f"{os.environ['INPUT_FOLDER']}{os.path.sep}hand-levels.csv", index=False)
